# 不依赖langchain的实现

## 需要安装的包

In [1]:
%%time
%%capture

!pip install faiss-gpu
!pip install sentence_transformers
!pip install openai
!pip install python-dotenv

CPU times: user 21.3 ms, sys: 19.3 ms, total: 40.5 ms
Wall time: 9.91 s


In [1]:
%%time

import os
from openai import OpenAI
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

CPU times: user 2.38 s, sys: 416 ms, total: 2.79 s
Wall time: 2.62 s


## 加载环境变量

In [2]:
%load_ext dotenv
%dotenv

token = os.environ.get('ONE_API_TOKEN')
base_url=os.environ.get('ONE_API_URL')

base_url

'http://192.168.0.72:3000/v1'

## 加载嵌入模型

In [3]:
%%time

encoder = SentenceTransformer("/models/bge-m3")

CPU times: user 1.67 s, sys: 1.12 s, total: 2.78 s
Wall time: 2.22 s


## 编造数据

In [4]:
data = [['《西游记》是中国古典小说四大名著之一，又称《大唐西游记》', '小说'],
['牛顿的主要著作是《数学原理》。这部著作于1687年首次出版，被认为是自然科学史上最重要的著作之一。', '数学'],
['东京迪斯尼乐园（Tokyo Disneyland）和东京迪士尼海洋（Tokyo DisneySea）是东京都内的两座迪士尼度假区。', '旅游'],
['电视连续剧《西游记》（1986年版）的主题曲是《云宫迅音》。', '音乐'],
['《西游记》主要内容围绕着僧人唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）西天取经的故事展开。他们历经九九八十一难，克服了各种艰险和困难，最终到达印度取回佛经。', '小说'],
['《西游记》电视连续剧讲述了唐僧师徒四人历经千辛万苦，斗妖除魔，取经西天的壮举，以及他们之间深厚的师徒情谊和对信仰与人性的思考。','电视剧']]
df = pd.DataFrame(data, columns = ['文本', '分类'])

df

,文本,分类
0,《西游记》是中国古典小说四大名著之一，又称《大唐西游记》,小说
1,牛顿的主要著作是《数学原理》。这部著作于1687年首次出版，被认为是自然科学史上最重要的著作之一。,数学
2,东京迪斯尼乐园（Tokyo Disneyland）和东京迪士尼海洋（Tokyo Disney...,旅游
3,电视连续剧《西游记》（1986年版）的主题曲是《云宫迅音》。,音乐
4,《西游记》主要内容围绕着僧人唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）西天取经的故事展开。...,小说
5,《西游记》电视连续剧讲述了唐僧师徒四人历经千辛万苦，斗妖除魔，取经西天的壮举，以及他们之间深...,电视剧


## 创建向量

简化处理，取消文档分片过程。

In [5]:
%%time

text = df['文本']
vectors = encoder.encode(text)

CPU times: user 475 ms, sys: 15.9 ms, total: 491 ms
Wall time: 498 ms


## 创建faiss索引

In [7]:
%%time

vector_dimension = vectors.shape[1] # 1024
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

CPU times: user 314 µs, sys: 0 ns, total: 314 µs
Wall time: 263 µs


## 创建查询向量

In [8]:
%%time

search_text = '西游记演了啥?'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

CPU times: user 30.1 ms, sys: 23.4 ms, total: 53.5 ms
Wall time: 56.4 ms


## 搜索最近相邻

In [9]:
%%time

distances, ann = index.search(_vector, k=3)

results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})

merge=pd.merge(results,df,left_on='ann',right_index=True)
merge

CPU times: user 3.38 ms, sys: 0 ns, total: 3.38 ms
Wall time: 3.18 ms


,distances,ann,文本,分类
0,0.571907,4,《西游记》主要内容围绕着僧人唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）西天取经的故事展开。...,小说
1,0.603106,5,《西游记》电视连续剧讲述了唐僧师徒四人历经千辛万苦，斗妖除魔，取经西天的壮举，以及他们之间深...,电视剧
2,0.698558,0,《西游记》是中国古典小说四大名著之一，又称《大唐西游记》,小说


In [10]:
merge['文本'].values

array(['《西游记》主要内容围绕着僧人唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）西天取经的故事展开。他们历经九九八十一难，克服了各种艰险和困难，最终到达印度取回佛经。',
       '《西游记》电视连续剧讲述了唐僧师徒四人历经千辛万苦，斗妖除魔，取经西天的壮举，以及他们之间深厚的师徒情谊和对信仰与人性的思考。',
       '《西游记》是中国古典小说四大名著之一，又称《大唐西游记》'], dtype=object)

## reranker

In [12]:
merged_items = [[search_text, x] for x in merge['文本'].values]
merged_items

[['西游记演了啥?',
  '《西游记》主要内容围绕着僧人唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）西天取经的故事展开。他们历经九九八十一难，克服了各种艰险和困难，最终到达印度取回佛经。'],
 ['西游记演了啥?',
  '《西游记》电视连续剧讲述了唐僧师徒四人历经千辛万苦，斗妖除魔，取经西天的壮举，以及他们之间深厚的师徒情谊和对信仰与人性的思考。'],
 ['西游记演了啥?', '《西游记》是中国古典小说四大名著之一，又称《大唐西游记》']]